# Car accidents in the US

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Show matplotlib's plots in Jupyter notebook
%matplotlib inline

In [2]:
accidents = pd.read_csv('US_Accidents_June20.csv')
# parse_dates=['Start_Time', 'End_Time', 'Weather_Timestamp']

In [3]:
accidents.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day


In [4]:
accidents.info()
# size: 1008.6+ MB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3513617 entries, 0 to 3513616
Data columns (total 49 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Source                 object 
 2   TMC                    float64
 3   Severity               int64  
 4   Start_Time             object 
 5   End_Time               object 
 6   Start_Lat              float64
 7   Start_Lng              float64
 8   End_Lat                float64
 9   End_Lng                float64
 10  Distance(mi)           float64
 11  Description            object 
 12  Number                 float64
 13  Street                 object 
 14  Side                   object 
 15  City                   object 
 16  County                 object 
 17  State                  object 
 18  Zipcode                object 
 19  Country                object 
 20  Timezone               object 
 21  Airport_Code           object 
 22  Weather_Timestamp 

# Duplicate values

In [5]:
accidents = accidents.drop_duplicates(subset=['Zipcode', 'Start_Time', 'End_Time'], keep='first')

# originele dataframe 3.513.617 rijen
# met deze dropped rijen 3.454.567 rows 

# Data types
check:
- [X] unieke values of ook rare waarden
- [X] missing values
 
33 - Bump                  - bool  - Schoon, alleen True en False\
34 - Crossing              - bool  - Schoon, alleen True en False\
35 - Give_Way              - bool  - Schoon, alleen True en False\
36 - Junction              - bool  - Schoon, alleen True en False\
37 - No_Exit               - bool  - Schoon, alleen True en False\
38 - Railway               - bool  - Schoon, alleen True en False\
39 - Roundabout            - bool  - Schoon, alleen True en False\
40 - Station               - bool  - Schoon, alleen True en False\
41 - Stop                  - bool  - Schoon, alleen True en False\
42 - Traffic_Calming       - bool  - Schoon, alleen True en False\
43 - Traffic_Signal        - bool  - Schoon, alleen True en False\
44 - Turning_Loop          - bool  - Alleen False --> verwijder variabele\
45 - Sunrise_Sunset        - object  - Night, Day, nan\
46 - Civil_Twilight        - object  - Night, Day, nan\
47 - Nautical_Twilight     - object  - Night, Day, nan\
48 - Astronomical_Twilight - object  - Night, Day, nan

In [6]:
#accidents_df['Severity'].unique()

In [7]:
# Change Severity to a category datatype.
#accidents['Severity'] = accidents['Severity'].astype('category')

#### Difference start time and end time

In [8]:
# Change Start_Time and End_Time to date-time variables
accidents['Start_Time'] = pd.to_datetime(accidents['Start_Time'])
accidents['End_Time'] = pd.to_datetime(accidents['End_Time'])
accidents['Weather_Timestamp'] = pd.to_datetime(accidents['Weather_Timestamp'])

In [9]:
# Create a new variable that measures total time of the accident. 
total_time = accidents['End_Time'] - accidents['Start_Time']

accidents.insert(loc=6, column='Total_Time', value=total_time)

In [10]:
#min(total_time)
#   '-1 days +23:25:55'
#max(total_time)
#   '987 days 11:15:29'


#accidents[accidents['Total_Time']>'30 days'].head()
#   22 rows waarbij de total time minder dan 0 is. 
#   Severity 2 en 3. 

#   326 rows waarbij de total time groter dan 30 dagen is.
#   Severity 2-3-4 (1 niet zichtbaar maar mogelijk wel erin).

In [11]:
# Drop all 22 rows where total time is less than 0
accidents = accidents[accidents['Total_Time']>'0']

In [12]:
# Create a new variable that changes total time of the accident to hours
my_list=[]

for obj in accidents['Total_Time']:
    my_list.append(int((obj.total_seconds())/60/60))  #Take the seconds of the deltatime object, then /60/60 to get hours
                
        
my_list

# Save as a new column in the data frame
#accidents.insert(loc=7, column='Total_Time_H', value=pd.Series(my_list))
accidents['Total_Time_H'] = my_list

In [13]:
accidents['Total_Time_H'].describe()

count    3.454545e+06
mean     1.304805e+00
std      4.694197e+01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      1.000000e+00
max      2.369900e+04
Name: Total_Time_H, dtype: float64

In [14]:
accidents['Total_Time_H'].isnull().sum()

0

#### Year of the accident

In [15]:
# Create a new variable for the year of the accidents (based off start year)
my_list=[]
for obj in accidents['Start_Time']:
    my_list.append(obj.year)
my_list

accidents.insert(loc=8, column='Year', value=my_list)

#### Month of the accident

In [16]:
# Create a new variable for the year of the accidents (based off start year)
my_list=[]
for obj in accidents['Start_Time']:
    my_list.append(obj.month)
my_list

accidents.insert(loc=8, column='Month', value=my_list)

In [17]:
accidents['Month'] = accidents['Month'].astype('category')

#### Hour of the day of the accident (based on Start_Time)

In [18]:
# Create a new variable for the year of the accidents (based off start year)
my_list=[]
for obj in accidents['Start_Time']:
    my_list.append(obj.hour)
my_list

accidents.insert(loc=8, column='Hour', value=my_list)

#### Day of the week

In [20]:
# create a week day variable
my_list = []

for day in accidents['Start_Time']:
    my_list.append(day.day_name())

weekday = pd.Series(my_list)

accidents.insert(loc=9, column='Weekday', value=weekday)

In [21]:
accidents['Weekday'] = accidents['Weekday'].astype('category')

#### Season of the accident

In [23]:
# get season from the dates. Months 1-3 are winter, 4-6 are spring, etc.
my_list=[]

for date in accidents['Start_Time']:
    if (date.month <= 2) | (date.month == 12):
        season = 'winter'
    elif (date.month >= 3) & (date.month <= 5):
        season = 'spring'
    elif (date.month >= 6) & (date.month <= 8):
        season = 'summer'
    elif (date.month >= 9) & (date.month <= 11):
        season = 'fall'
    else:
        season = 'nothing???'
    my_list.append(season)

season = pd.Series(my_list)

accidents.insert(loc=10, column='Season', value=season)
#accidents[accidents['Season']] = season

In [24]:
season.unique()

array(['winter', 'spring', 'summer', 'fall'], dtype=object)

#### Create a new value for the sunset_sunrise: dusk/twilight
Create a new value for dusk/twilight using the other light columns. Vision is limited during twilight compared to during the day, but vision is better than during the night. 

In [25]:
light = []
error = 0

for i, value in accidents['Sunrise_Sunset'].items():
    if value == 'Day':
        light.append('Day')
    elif value == 'Night':
        if accidents['Astronomical_Twilight'].loc[i] == 'Day':
            light.append('Twilight')
        elif accidents['Astronomical_Twilight'].loc[i] == 'Night':
            light.append('Night')
    else:
        error += 1
        light.append(float('NaN'))
        # during nan values? count number of errors 
        
# Create a new variable in 'accidents' dataframe
accidents['Light'] = light

In [165]:
# number of missing values
error
accidents['Light'].isna().sum()

110

In [28]:
# alle tijen met missing data in de accidents['Light'] kolom
mask_nan_light = accidents['Light'].isna()

accidents[mask_nan_light]

,ID,Source,TMC,Severity,Start_Time,End_Time,Total_Time,Start_Lat,Hour,Weekday,...,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Total_Time_H,Light
85973,A-85975,MapQuest,229.0,2,2016-08-02 18:18:02,2016-08-02 19:03:02,0 days 00:45:00,34.451862,18,Tuesday,...,False,False,False,False,NaN,NaN,NaN,NaN,0,NaN
111085,A-111087,MapQuest,201.0,2,2016-06-01 10:26:08,2016-06-01 11:10:00,0 days 00:43:52,34.451862,10,Wednesday,...,False,False,False,False,NaN,NaN,NaN,NaN,0,NaN
119777,A-119779,MapQuest,201.0,2,2016-12-23 19:30:59,2016-12-23 20:15:41,0 days 00:44:42,27.388653,19,Sunday,...,False,False,True,False,NaN,NaN,NaN,NaN,0,NaN
122934,A-122936,MapQuest,201.0,2,2017-01-17 17:34:09,2017-01-17 18:04:09,0 days 00:30:00,27.387951,17,Wednesday,...,False,False,False,False,NaN,NaN,NaN,NaN,0,NaN
123707,A-123709,MapQuest,201.0,2,2017-01-24 07:30:44,2017-01-24 08:15:29,0 days 00:44:45,27.388653,7,Tuesday,...,False,False,True,False,NaN,NaN,NaN,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328098,A-3328261,Bing,NaN,2,2018-04-09 19:18:06,2018-04-10 01:18:06,0 days 06:00:00,41.036740,19,Tuesday,...,False,False,False,False,NaN,NaN,NaN,NaN,6,NaN
3377672,A-3377835,Bing,NaN,4,2017-11-29 11:51:23,2017-11-30 05:51:23,0 days 18:00:00,41.152440,11,Wednesday,...,False,False,False,False,NaN,NaN,NaN,NaN,18,NaN
3383498,A-3383662,Bing,NaN,2,2017-12-13 18:38:56,2017-12-14 00:38:56,0 days 06:00:00,38.965710,18,Wednesday,...,False,False,True,False,NaN,NaN,NaN,NaN,6,NaN
3400065,A-3400229,Bing,NaN,4,2017-11-24 04:48:21,2017-11-24 10:48:21,0 days 06:00:00,40.586870,4,Wednesday,...,False,False,False,False,NaN,NaN,NaN,NaN,6,NaN


In [149]:
month_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11 , 12]

def get_minmax_time_daytwilight(list_months):
    all_months = []
    for month in list_months:
        #maak voor elke maand een apart dataframe aan
        accidents_month = accidents[accidents['Month'] == month]
        
        # ga kijken naar de max en min waarden van daglicht voor deze maand en sla deze op in my_list
        list_times_day_night = [month]
        
        day_min = min(accidents_month[accidents_month['Light'] == 'Day']['Start_Time'])
        list_times_day_night.append(day_min)
        day_max = max(accidents_month[accidents_month['Light'] == 'Day']['Start_Time'])
        list_times_day_night.append(day_max)
        
        twilight_min = min(accidents_month[accidents_month['Light'] == 'Twilight']['Start_Time'])
        list_times_day_night.append(twilight_min)
        twilight_max = max(accidents_month[accidents_month['Light'] == 'Twilight']['Start_Time'])
        list_times_day_night.append(twilight_max)
        
        # put this list in an overall list.
        all_months.append(list_times_day_night)
        
    # turn the list of all month's data into a dataframe. print
    all_months_df = pd.DataFrame(all_months)
    all_months_df = all_months_df.rename(columns = {0:'month', 1:'min_day', 2:'max_day',
                                                    3:'min_twilight', 4:'max_twilight'})
    return all_months_df    
    
df = get_minmax_time_daytwilight(month_list)
df = df.set_index('month')

In [162]:
# a function to fill up missing values of 'Light' based on known values

def fill_nans_light(dataframe_accidents, dataframe_times_minmax):
    new_series = []
    
    for i, value in dataframe_accidents['Light'].items():
        if pd.isna(value) == False:
            new_series.append(value)
            # if the value of the light series is NOT missing, then just put that value in the next list
            
        else:
            nan_month = dataframe_accidents['Month'].iloc[i]

            # get the times on which the state of 'light' changes (e.g. from 'Twilight' to 'Day')
            min_day = dataframe_times_minmax.at[nan_month, 'min_day'].time()
            max_day = dataframe_times_minmax.at[nan_month, 'max_day'].time()
            min_twilight = dataframe_times_minmax.at[nan_month, 'min_twilight'].time()
            max_twilight = dataframe_times_minmax.at[nan_month, 'max_twilight'].time()
            
            time_of_day_nan = dataframe_accidents['Start_Time'].iloc[i].time()
            
            #print(f"maand:{nan_month}, min:{min_day}, max:{max_day}, werkelijk:{time_of_day_nan}")
            
            # check the time.
            if (time_of_day_nan >= min_day) and (time_of_day_nan <= max_day):
                new_series.append('Day')
            elif (time_of_day_nan >= min_twilight) and (time_of_day_nan < min_day):
                # eerste twilight ('s ochtends)
                new_series.append('Twilight')
            elif (time_of_day_nan <= max_twilight) and (time_of_day_nan > max_day):
                # tweede twilight ('s avonds)
                new_series.append('Twilight')
            elif (time_of_day_nan < min_twilight) and (time_of_day_nan > max_twilight):
                new_series.append('Night')
            else:
                new_series.append(float('nan'))
    
    # turn the list into a series that we can insert into the dataframe. Return it so it can be saved.
    new_series = pd.Series(new_series)
    return new_series


new_light_series = fill_nans_light(accidents, df)

In [163]:
new_light_series.isna().sum()

16

In [152]:
new_light_series.unique()

array(['Night', 'Twilight', 'Day', nan], dtype=object)

In [157]:
mask = new_light_series.isna()
list_indices = list(new_light_series[mask].index)
list_indices

[122896,
 123669,
 640144,
 650539,
 2093273,
 2132193,
 2510158,
 2518661,
 2547987,
 2677267,
 2796191,
 2848033,
 2911254,
 2990182,
 3205606,
 3325933]

In [158]:
accidents.iloc[list_indices]

,ID,Source,TMC,Severity,Start_Time,End_Time,Total_Time,Start_Lat,Hour,Weekday,...,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight,Total_Time_H,Light
122934,A-122936,MapQuest,201.0,2,2017-01-17 17:34:09,2017-01-17 18:04:09,0 days 00:30:00,27.387951,17,Wednesday,...,False,False,False,False,NaN,NaN,NaN,NaN,0,NaN
123707,A-123709,MapQuest,201.0,2,2017-01-24 07:30:44,2017-01-24 08:15:29,0 days 00:44:45,27.388653,7,Tuesday,...,False,False,True,False,NaN,NaN,NaN,NaN,0,NaN
640698,A-640710,MapQuest,201.0,4,2020-03-03 18:47:01,2020-03-03 21:05:37,0 days 02:18:36,39.129887,18,Saturday,...,False,False,False,False,NaN,NaN,NaN,NaN,2,NaN
651103,A-651116,MapQuest,201.0,2,2020-01-10 18:27:18,2020-01-10 18:56:36,0 days 00:29:18,38.849342,18,Friday,...,False,False,False,False,NaN,NaN,NaN,NaN,0,NaN
2096374,A-2096414,MapQuest,201.0,3,2018-02-23 11:34:25,2018-02-23 12:03:53,0 days 00:29:28,27.388699,11,Saturday,...,False,False,False,False,NaN,NaN,NaN,NaN,0,NaN
2135329,A-2135369,MapQuest-Bing,201.0,2,2018-01-16 16:36:57,2018-01-16 17:06:35,0 days 00:29:38,38.876060,16,Wednesday,...,False,False,False,False,NaN,NaN,NaN,NaN,0,NaN
2514991,A-2515032,Bing,NaN,4,2016-12-11 16:33:08,2016-12-11 22:33:08,0 days 06:00:00,41.036740,16,Sunday,...,False,False,False,False,NaN,NaN,NaN,NaN,6,NaN
2524061,A-2524102,Bing,NaN,4,2017-01-10 09:42:10,2017-01-10 15:42:10,0 days 06:00:00,40.843090,9,Monday,...,False,False,False,False,NaN,NaN,NaN,NaN,6,NaN
2555509,A-2555550,Bing,NaN,2,2016-08-14 07:20:11,2016-08-14 13:20:11,0 days 06:00:00,41.036740,7,Thursday,...,False,False,False,False,NaN,NaN,NaN,NaN,6,NaN
2692622,A-2692670,Bing,NaN,2,2017-06-05 06:17:40,2017-06-05 12:17:40,0 days 06:00:00,41.036740,6,Thursday,...,False,False,False,False,NaN,NaN,NaN,NaN,6,NaN


#### Create a new column: total number of obstructions reported. 

In [ ]:
# dit kan makkelijker mischien door de lijst in te vullen, dan heb je niet al de herhalingen onderin.
obst = ['Bump', 'Crossing', 'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout',
        'Station', 'Stop', 'Traffic_Calming', 'Traffic_Signal']

def equals_true(series):
    my_list=[]
    for x in series:
        if x == True:
            my_list.append(1)
        else:
            my_list.append(0)
            
    # list_of_series:
    
    return pd.Series(my_list)
        
obst_bump = equals_true(accidents['Bump'])
obst_cros = equals_true(accidents['Crossing'])
obst_give = equals_true(accidents['Give_Way'])
obst_junc = equals_true(accidents['Junction'])
obst_noex = equals_true(accidents['No_Exit'])
obst_rail = equals_true(accidents['Railway'])
obst_roun = equals_true(accidents['Roundabout'])
obst_stat = equals_true(accidents['Station'])
obst_stop = equals_true(accidents['Stop'])
obst_tr_c = equals_true(accidents['Traffic_Calming'])
obst_tr_s = equals_true(accidents['Traffic_Signal'])

n_obst = (obst_bump + obst_cros + obst_give + obst_junc + obst_noex + obst_rail +
          obst_roun+ obst_stat + obst_stop + obst_tr_c + obst_tr_s)

In [ ]:
accidents['Total_Obstr'] = n_obst

#### Create a new column: no obstructions reported

In [ ]:
zero_obstr = []

for value in n_obst:
    if value == 0:
        zero_obstr.append(True)
    else:
        zero_obstr.append(False)

In [ ]:
accidents['No_Obstr'] = my_list

#### Drop rows where the weather information was recorded >1 hour before or after the accident

In [ ]:
# Ilse 
accidents["Diff_h_wt"] = accidents["Start_Time"] - accidents["Weather_Timestamp"] 
accidents["Diff_h_wt"] = accidents["Diff_h_wt"] / pd.Timedelta(hours=1)
accidents = accidents.drop(accidents[(accidents.Diff_h_wt < -1)].index)
accidents = accidents.drop(accidents[(accidents.Diff_h_wt > 1)].index)

In [ ]:
accidents['Season'] = accidents['Season'].astype('category')